<a href="https://colab.research.google.com/github/elizabeth-evolkmar/Trabajo-parqueadero-udea/blob/main/Codigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import sys
import time
import platform
import getpass
from datetime import datetime, timedelta

# ------------------------------
# Constantes del sistema
# ------------------------------
MAX_SPACES = 64  # Máximo de espacios disponibles en el parqueadero
HOURLY_RATE = 7000  # Tarifa por hora
QUARTER_HOUR_RATE = 1500  # Tarifa por cada 15 minutos
LOG_FILE = 'event_log.txt'  # Archivo donde se guarda el log de eventos
CSV_FILE = 'report.csv'  # Archivo CSV de reportes

# ------------------------------
# Bases de datos en memoria
# ------------------------------
db_users = {}  # Diccionario de usuarios registrados
parked = {}  # Diccionario de vehículos actualmente parqueados
history = []  # Historial de entradas y salidas
procedures_count = []  # Registro de eventos para auditoría

# ------------------------------
# Usuarios con rol definido
# ------------------------------
admins = {'admin': 'admin123'}
cashiers = {'cajero': 'cajero123'}

# ------------------------------
# Función para registrar eventos
# ------------------------------
def log_event(action):
    now = datetime.now()
    duration = 0
    header = f"User:{getpass.getuser()} | OS:{platform.system()} {platform.release()} | Platform:{platform.platform()}"
    timestamp = now.strftime('%Y-%m-%d %H:%M:%S.%f')
    with open(LOG_FILE, 'a') as f:
        f.write(header + '\n')
        f.write(f"{timestamp}|{action}|{duration:.6f}s\n")
    procedures_count.append(action)

# ------------------------------
# Validaciones básicas
# ------------------------------
def validate_name(name):
    return name.isalpha() and len(name) >= 3

def validate_document(doc):
    return doc.isdigit() and 3 <= len(doc) <= 15

def validate_plate(plate):
    return len(plate) == 6 and plate[:3].isalpha() and plate[3:].isdigit()

# ------------------------------
# Registro de usuario
# ------------------------------
def register_user():
    print("-- Registro de Usuario --")
    name = input("Nombre: ")
    if not validate_name(name):
        print("Error: Nombre inválido.")
        return
    surname = input("Apellido: ")
    if not validate_name(surname):
        print("Error: Apellido inválido.")
        return
    doc = input("Documento: ")
    if not validate_document(doc):
        print("Error: Documento inválido.")
        return
    plate = input("Placa (3 letras y 3 números): ")
    if not validate_plate(plate):
        print("Error: Placa inválida.")
        return
    for u in db_users.values():
        if u['plate'].lower() == plate.lower():
            print("Error: Placa ya registrada.")
            return
    db_users[doc] = {'name': name, 'surname': surname, 'plate': plate.upper()}
    print("Usuario registrado con éxito.")
    log_event('register_user')

# ------------------------------
# Ingreso de vehículo
# ------------------------------
def enter_vehicle():
    print("-- Ingreso de Vehículo --")
    plate = input("Ingrese placa: ").upper()

    user = None
    for datos in db_users.values():
        if datos['plate'].upper() == plate:
            user = datos
            break

    if not user:
        print("Error: Usuario no registrado.")
        return

    if len(parked) >= MAX_SPACES:
        print("Error: No hay espacios disponibles.")
        return

    entry_time = datetime.now()
    parked[plate] = entry_time
    history.append({'plate': plate, 'entry': entry_time, 'exit': None, 'fee': None})
    print(f"Vehículo {plate} ingresado a las {entry_time.strftime('%Y-%m-%d %H:%M:%S')}")
    log_event('enter_vehicle')

# ------------------------------
# Retiro de vehículo
# ------------------------------
def exit_vehicle():
    print("-- Retiro de Vehículo --")
    plate = input("Placa del vehículo: ").upper()
    entry_time = parked.get(plate)
    if not entry_time:
        print("Error: Vehículo no está en el parqueadero.")
        return
    exit_time = datetime.now()
    delta = exit_time - entry_time
    hours = delta.seconds // 3600
    quarters = (delta.seconds % 3600) // 900
    total = max(hours * HOURLY_RATE + quarters * QUARTER_HOUR_RATE, HOURLY_RATE)
    print(f"Total a pagar: {total} pesos")
    for rec in history:
        if rec['plate'] == plate and rec['exit'] is None:
            rec['exit'] = exit_time
            rec['fee'] = total
            break
    del parked[plate]
    log_event('exit_vehicle')

# ------------------------------
# Cierre de caja
# ------------------------------
def cierre_de_caja():
    print("-- Cierre de Caja --")
    total_payments = sum(r['fee'] or 0 for r in history)
    print(f"Total recaudo hasta ahora: {total_payments} pesos")
    log_event('cierre_de_caja')

# ------------------------------
# Exportar historial a CSV
# ------------------------------
def export_csv():
    with open(CSV_FILE, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['plate', 'entry', 'exit', 'fee'])
        writer.writeheader()
        for r in history:
            writer.writerow({
                'plate': r['plate'],
                'entry': r['entry'].strftime('%Y-%m-%d %H:%M:%S'),
                'exit': r['exit'].strftime('%Y-%m-%d %H:%M:%S') if r['exit'] else '',
                'fee': r['fee'] or ''
            })
    print(f"Reporte exportado a {CSV_FILE}")
    log_event('export_csv')

# ------------------------------
# Estadísticas del parqueadero
# ------------------------------
def estadisticas_parqueadero():
    registered = len(set(r['plate'] for r in history))
    retired = sum(1 for r in history if r['exit'] is not None)
    no_retired = sum(1 for r in history if r['exit'] is None)
    total_paid = sum(r['fee'] or 0 for r in history)

    time = [
        (r['exit'] - r['entry']).total_seconds()
        for r in history if r['exit'] and r['entry']
    ]
    average_time = sum(time) / len(time) if time else 0
    average_time_min = round(average_time / 60, 2)

    vehicle_max = vehicle_min = None
    if time:
        vehicle_max = max(
            (r for r in history if r['exit']),
            key=lambda r: (r['exit'] - r['entry']).total_seconds()
        )
        vehicle_min = min(
            (r for r in history if r['exit']),
            key=lambda r: (r['exit'] - r['entry']).total_seconds()
        )

    with open(CSV_FILE, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([])
        writer.writerow(['---- ESTADÍSTICAS ----'])
        writer.writerow(['Total vehículos registrados (únicos)', registered])
        writer.writerow(['Total vehículos retirados', retired])
        writer.writerow(['Total vehículos sin retirar', no_retired])
        writer.writerow(['Total pagos recibidos', total_paid])
        writer.writerow(['Tiempo promedio de estancia (min)', average_time_min])
        if vehicle_max and vehicle_min:
            writer.writerow(['Vehículo con mayor tiempo parqueado', vehicle_max['plate']])
            writer.writerow(['Vehículo con menor tiempo parqueado', vehicle_min['plate']])

    print("Estadísticas exportadas a report.csv")
    log_event('estadisticas_parqueadero')

# ------------------------------
# Exportar usuarios a CSV
# ------------------------------
def exportar_usuarios():
    with open('users.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['documento', 'nombre', 'apellido', 'placa'])
        writer.writeheader()
        for doc, data in db_users.items():
            writer.writerow({
                'documento': doc,
                'nombre': data['name'],
                'apellido': data['surname'],
                'placa': data['plate']
            })
    print("Usuarios exportados a users.csv")
    log_event('exportar_usuarios')

# ------------------------------
# Login de usuario
# ------------------------------
def login(role_store, role_name):
    user = input(f"Usuario {role_name}: ")
    pwd = input("Contraseña: ")
    if role_store.get(user) == pwd:
        return True
    print("Credenciales incorrectas.")
    return False

# ------------------------------
# Menú del cajero
# ------------------------------
def cashier_menu():
    while True:
        print("\n-- Menú Cajero --")
        print("1. Registrar Usuario")
        print("2. Ingresar Vehículo")
        print("3. Retirar Vehículo")
        print("0. Cerrar sesión")
        c = input("Opción: ")
        if c == '1':
            register_user()
        elif c == '2':
            enter_vehicle()
        elif c == '3':
            exit_vehicle()
        elif c == '0':
            break
        else:
            print("Opción inválida.")

# ------------------------------
# Menú del administrador
# ------------------------------
def admin_menu():
    while True:
        print("\n-- Menú Administrador --")
        print("1. Cierre de Caja")
        print("2. Exportar Reporte CSV")
        print("3. Ver estadísticas generales")
        print("4. Exportar lista de usuarios")
        print("0. Cerrar sesión")
        c = input("Opción: ")
        if c == '1':
            cierre_de_caja()
        elif c == '2':
            export_csv()
        elif c == '3':
            estadisticas_parqueadero()
        elif c == '4':
            exportar_usuarios()
        elif c == '0':
            break
        else:
            print("Opción inválida.")

# ------------------------------
# Función principal
# ------------------------------
def main():
    print(r"""
              ______
           __/  |   \___
          |  _     _  ``-.
     _____| (_)___(_)____|__
    """)
    print("---- Bienvenido parking express ----")
    while True:
        role = input("¿Ingresar como admin o cajero? (admin/cajero/exit): ")
        if role == 'admin':
            if login(admins, 'admin'):
                admin_menu()
        elif role == 'cajero':
            if login(cashiers, 'cajero'):
                cashier_menu()
        elif role == 'exit':
            print("Saliendo...")
            sys.exit()
        else:
            print("Rol inválido.")

if __name__ == '__main__':
    main()

---- Sistema Parqueadero ----
¿Ingresar como admin o cajero? (admin/cajero/exit): cajero
Usuario cajero: cajero
Contraseña: cajero123

-- Menú Cajero --
1. Registrar Usuario
2. Ingresar Vehículo
3. Retirar Vehículo
0. Cerrar sesión
Opción: 1
-- Registro de Usuario --
Nombre: suata
Apellido: alas
Documento: 1234
Placa (3 letras y 3 números): 2355
Error: Placa inválida.

-- Menú Cajero --
1. Registrar Usuario
2. Ingresar Vehículo
3. Retirar Vehículo
0. Cerrar sesión
Opción: 0
¿Ingresar como admin o cajero? (admin/cajero/exit): exit
Saliendo...


# Manual de Usuario
## Sistema de Gestión Parqueadero Parking Express

### Información del Proyecto
**Versión:** 1.0  
**Fecha:** Junio 2025  
**Desarrollado por:** Elizabeth Escobar, Tatiana Carvajal, Luis Eduar León  

---

## 1. Introducción

### 1.1 ¿Qué es el Sistema Parking Express?
El Sistema de Gestión Parqueadero Parking Express es una aplicación desarrollada en Python que automatiza el control de entrada y salida de vehículos en el parqueadero de la Universidad de Antioquia.

### 1.2 Características Principales
- Registro de usuarios y vehículos
- Control de ingreso y salida de vehículos
- Cálculo automático de tarifas
- Generación de reportes administrativos
- Exportación de datos en formato CSV
- Sistema de autenticación por roles

### 1.3 Capacidad del Sistema
- **Espacios disponibles:** 64 vehículos máximo
- **Tarifas:** $7,000 por hora, $1,500 por cuarto de hora
- **Pago mínimo:** $7,000 pesos

---

## 2. Requisitos del Sistema

### 2.1 Requisitos Técnicos
- Python 3.x instalado
- Bibliotecas necesarias: `csv`, `datetime`, `getpass`, `platform`
- Sistema operativo: Windows, macOS o Linux

### 2.2 Archivos Generados por el Sistema
- `event_log.txt`: Registro de todas las actividades
- `report.csv`: Reporte exportable de operaciones

---

## 3. Inicio del Sistema

### 3.1 Ejecutar el Programa
1. Abra su terminal o línea de comandos
2. Navegue hasta la carpeta donde está el archivo del programa
3. Ejecute el comando: `python codigo.py`
4. Aparecerá la pantalla de bienvenida

### 3.2 Pantalla de Inicio
```
---- Sistema Parqueadero ----
¿Ingresar como admin o cajero? (admin/cajero/exit):
```

**Opciones disponibles:**
- `admin`: Acceso administrativo
- `cajero`: Acceso operativo
- `exit`: Salir del sistema

---

## 4. Guía para Cajeros

### 4.1 Inicio de Sesión como Cajero
1. Escriba `cajero` en la pantalla principal
2. Ingrese las credenciales:
   - **Usuario:** `cajero`
   - **Contraseña:** `cajero123`

### 4.2 Menú Principal del Cajero
```
-- Menú Cajero --
1. Registrar Usuario
2. Ingresar Vehículo
3. Retirar Vehículo
0. Cerrar sesión
```

### 4.3 Registrar Usuario

#### Paso a Paso:
1. Seleccione opción `1`
2. Complete los siguientes datos:

**Nombre:**
- Mínimo 3 letras
- Solo letras (sin números ni símbolos)
- Ejemplo: `Juan`

**Apellido:**
- Mínimo 3 letras
- Solo letras (sin números ni símbolos)
- Ejemplo: `Pérez`

**Documento:**
- Entre 3 y 15 dígitos
- Solo números
- Ejemplo: `1234567890`

**Placa del Vehículo:**
- Exactamente 6 caracteres
- Formato: 3 letras + 3 números
- Ejemplo: `ABC123`

#### Mensajes de Error Comunes:
- "Error: Nombre inválido" - Revise que contenga solo letras y mínimo 3 caracteres
- "Error: Documento inválido" - Verifique que sean solo números entre 3 y 15 dígitos
- "Error: Placa inválida" - Confirme el formato: 3 letras seguidas de 3 números

### 4.4 Ingresar Vehículo

#### Paso a Paso:
1. Seleccione opción `2`
2. Ingrese el documento del usuario registrado
3. El sistema mostrará la hora de ingreso

#### Posibles Situaciones:
- **Usuario no registrado:** "Error: Usuario no registrado"
- **Parqueadero lleno:** "Error: No hay espacios disponibles"
- **Ingreso exitoso:** Se muestra placa y hora de entrada

### 4.5 Retirar Vehículo

#### Paso a Paso:
1. Seleccione opción `3`
2. Ingrese la placa del vehículo
3. El sistema calculará automáticamente el costo
4. Se mostrará el total a pagar

#### Cálculo de Tarifas:
- **Tarifa por hora:** $7,000
- **Tarifa por cuarto de hora:** $1,500
- **Mínimo a pagar:** $7,000

#### Ejemplo de Cálculo:
- Tiempo: 2 horas y 45 minutos
- Cálculo: 2 × $7,000 + 3 × $1,500 = $18,500

---

## 5. Guía para Administradores

### 5.1 Inicio de Sesión como Administrador
1. Escriba `admin` en la pantalla principal
2. Ingrese las credenciales:
   - **Usuario:** `admin`
   - **Contraseña:** `admin123`

### 5.2 Menú Principal del Administrador
```
-- Menú Administrador --
1. Cierre de Caja
2. Exportar Reporte CSV
0. Cerrar sesión
```

### 5.3 Cierre de Caja
- Seleccione opción `1`
- El sistema mostrará el total recaudado hasta el momento
- Información mostrada: "Total recaudo hasta ahora: $X pesos"

### 5.4 Exportar Reporte CSV
- Seleccione opción `2`
- Se genera el archivo `report.csv`
- Contiene: placa, hora de entrada, hora de salida, tarifa cobrada

#### Estructura del Archivo CSV:
```csv
plate,entry,exit,fee
ABC123,2025-06-15 10:30:00,2025-06-15 12:45:00,18500
DEF456,2025-06-15 11:00:00,,
```

---

## 6. Validaciones y Restricciones

### 6.1 Validaciones de Datos

| Campo | Reglas | Ejemplo Válido | Ejemplo Inválido |
|-------|--------|----------------|------------------|
| Nombre | Mínimo 3 letras, solo letras | "Carlos" | "Ca", "Car123" |
| Apellido | Mínimo 3 letras, solo letras | "González" | "Go", "Gon456" |
| Documento | 3-15 dígitos, solo números | "1234567890" | "12", "abc123" |
| Placa | 6 caracteres: 3 letras + 3 números | "ABC123" | "AB123", "ABCD12" |

### 6.2 Restricciones Operativas
- Máximo 64 vehículos simultáneos
- Solo usuarios registrados pueden ingresar vehículos
- Solo vehículos ingresados pueden ser retirados
- Un usuario puede tener solo un vehículo registrado

---

## 7. Mensajes del Sistema

### 7.1 Mensajes de Éxito
- "Usuario registrado con éxito"
- "Vehículo [placa] ingresado a las [fecha y hora]"
- "Total a pagar: $X pesos"
- "Reporte exportado a report.csv"

### 7.2 Mensajes de Error
- "Credenciales incorrectas"
- "Error: Usuario no registrado"
- "Error: Vehículo no está en el parqueadero"
- "Error: No hay espacios disponibles"
- "Opción inválida"

---

## 8. Resolución de Problemas

### 8.1 Problemas Comunes

**Problema:** No puedo registrar un usuario
- **Solución:** Verifique que todos los datos cumplan con las validaciones

**Problema:** El vehículo no aparece para retiro
- **Solución:** Confirme que la placa esté correctamente escrita y que el vehículo haya ingresado

**Problema:** No puedo acceder como administrador/cajero
- **Solución:** Verifique las credenciales exactas (distingue mayúsculas y minúsculas)

### 8.2 Contacto de Soporte
Para soporte técnico, contacte al equipo de desarrollo:
- Elizabeth Escobar (Líder del proyecto)
- Tatiana Carvajal
- Luis Eduar León

---

## 9. Anexos

### 9.1 Credenciales del Sistema
**Cajero:**
- Usuario: `cajero`
- Contraseña: `cajero123`

**Administrador:**
- Usuario: `admin`
- Contraseña: `admin123`

### 9.2 Archivos de Registro
- **event_log.txt:** Registro detallado de todas las operaciones
- **report.csv:** Reporte exportable de movimientos del parqueadero

### 9.3 Licencia
Este software está bajo licencia Creative Commons Atribución-NoComercial-SinDerivadas 4.0 Internacional (CC BY-NC-ND 4.0).

---

**© 2025 - Sistema Parking Express UdeA**  
*Desarrollado por estudiantes de Ingeniería Industrial - Universidad de Antioquia*